<a href="https://colab.research.google.com/github/ibrunaneves/PraticasDeMachineLearning/blob/main/Q2_Hackathon_Unifacisa_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instituição - UNIFACISA
# Autor: Bruna Neves - 27/03/25

**Questão 2 (intermediário)**

Detecção de Fraudes em Cartões de Crédito O dataset de transações financeiras contém colunas como valor da transação, localização, histórico do usuário, horário da compra, entre outras.

Seu objetivo é criar um modelo que identifique possíveis fraudes com base nesses dados.

Tarefas: Use um dataset de fraudes em cartão de crédito (pode ser obtido no Kaggle ou gerado artificialmente).

Faça a pré-processamento dos dados, lidando com valores nulos e normalizando as colunas numéricas. Escolha um modelo adequado (exemplo: Árvores de Decisão, Random Forest, ou XGBoost).

Treine o modelo e avalie seu desempenho com métricas como precisão, recall e F1-score.

Pergunta: O modelo teve um bom desempenho na detecção de fraudes?
Que técnicas poderiam ser aplicadas para melhorar a classificação?

In [ ]:
# Importando as bibliotecas de dados e ML
import numpy as np # biblioteca que trabalha com dados, vetores e matrizes multidimensionais
import pandas as pd # biblioteca que faz tudo que o excel faz
from sklearn.datasets import make_classification # carrega uma base de dados de classificação de formas geométricas

from sklearn.model_selection import train_test_split # sepera a base em treino e teste (70% e 30%)
from sklearn.preprocessing import StandardScaler # biblioteca usada para fazer normalização dos dados
from sklearn.ensemble import RandomForestClassifier # algoritmo de classificação
from sklearn.metrics import classification_report # relatório geral com todas as métricas de classificação

In [ ]:
# Parte 1 - gerando a massa de dados com exemplos de estrutura de cartões de crédito (com dados sintéticos)

In [ ]:
# Definindo o número de amostras
total_samples = 10000 # total de exemplos
fraud_ratio = 0.05 # 5% de fraudes
fraud_samples = int(total_samples * fraud_ratio)
normal_samples = total_samples - fraud_samples

In [ ]:
# Gerar base de dados sintéticos com make_classification
X, y = make_classification(n_samples=total_samples, n_features=5, weights=[0.95, 0.05], random_state=42)

In [ ]:
# Criar o DataFramepd.DataFrame()
df = pd.DataFrame(X, columns=['valor_transacao', 'latitude', 'longitude', 'historico_usuario', 'horario_compra'])
df['fraude'] = y

In [ ]:
# Exibindo as primeiras linhdas do DataFrame
df.head(100)

,valor_transacao,latitude,longitude,historico_usuario,horario_compra,fraude
0,-2.244569,-1.362328,1.554333,-2.086909,-1.277605,0
1,-0.741786,0.204282,1.681893,-2.608295,1.980196,0
2,-1.817423,-1.437839,0.661023,-0.708437,0.727114,0
3,-1.122643,0.106464,2.183630,-3.353276,-0.719814,0
4,-0.018329,1.014871,1.843978,-3.024641,-0.478505,0
...,...,...,...,...,...,...
95,-0.351436,0.049514,0.712461,-1.097168,1.593106,0
96,-0.075616,0.928005,1.790664,-2.925192,-1.114507,0
97,-2.327344,-1.842930,0.843507,-0.902308,1.761403,0
98,0.322783,0.230613,-0.161582,0.198383,0.533520,0


In [ ]:
# Ajuste de valores
def ajustar_valores(df):
    df['valor_transacao'] = np.abs(df['valor_transacao']) * 1000  # Converter para valores positivos
    df['latitude'] = np.round(np.random.uniform(-90, 90, total_samples), 6)
    df['longitude'] = np.round(np.random.uniform(-180, 180, total_samples), 6)
    df['historico_usuario'] = np.random.randint(1, 50, total_samples)  # Número de transações anteriores
    df['horario_compra'] = np.random.randint(0, 24, total_samples)     # Horário do dia
    return df


In [ ]:
# Adicionando ele ao DataFrame
df = ajustar_valores(df)
# exibindo as primeiras 10 linhas
df.head(10)

,valor_transacao,latitude,longitude,historico_usuario,horario_compra,fraude
0,2244.569342,-25.309260,129.269967,3,0,0
1,741.785718,-48.450860,-36.152502,49,1,0
2,1817.422978,-53.817217,-43.114212,26,16,0
3,1122.642954,-76.154729,50.659369,8,9,0
4,18.328603,-82.991538,69.946064,32,5,0
5,2283.124880,24.353912,62.861263,40,11,0
6,1309.595565,26.500886,-57.586331,43,13,0
7,1153.680358,20.817993,-29.103720,44,7,0
8,1619.439194,-30.086397,46.357327,44,3,0
9,1310.700403,67.153315,46.051804,31,5,0


In [ ]:
# Salvar os dados em um arquivo CSV
df.to_csv('fraudes_cartao.csv', index=False)
print("Base de dados gerada com sucesso!")

Base de dados gerada com sucesso!


In [ ]:
# Parte 2 - Gerando modelo de ML para identificar fraudes em cartões de crédito

In [ ]:
# Pré-processamento dos dados
df.fillna(df.mean(), inplace=True) # serve para lidar com valores nulos
# aplicar a normalização entre os dados
scaller = StandardScaler()
df[['valor_transacao', 'historico_usuario', 'horario_compra']] = scaller.fit_transform(df[['valor_transacao', 'historico_usuario', 'horario_compra']])

In [ ]:
# Separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('fraude', axis=1), df['fraude'], test_size=0.2, random_state=42)

In [ ]:
# Escolher modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Avaliação
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1905
           1       0.00      0.00      0.00        95

    accuracy                           0.95      2000
   macro avg       0.48      0.50      0.49      2000
weighted avg       0.91      0.95      0.93      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
